In [1]:

# Make syre these libraries are installed: torch, torchvision, tqdm, matplotlib
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
import re
import os
from torchvision.transforms import RandomRotation
from torchvision.io import read_image
from tqdm import * # Progress Bars
import matplotlib.pyplot as plt

# These are just other files
from utils import *
from models import *


In [16]:
device = 'mps'  
# For device, use 'cpu' to run on your cpu, 'mps' runs on the macbook graphics card, if you have cuda, you can also do 'cuda' and it will be very fast

data = ASL_Dataset(transform=None, img_shape=64)  # For now, keep img_shape = 64, I havent had time to deal with the model handling other shapes.  
                                                  # This is fine for now we can change it later if needed.
                                                  # For transform, this is for any of the RandomRotation, etc. from torchvision
model = ASL_Classifier(dropout_rate=0.25, device = device)


100%|██████████| 29/29 [00:00<00:00, 188.82it/s]


splitting data

In [17]:
train_proportion = 0.8 # Should definitely be much higher, just wanted to see with small data
val_proportion = 0.05
test_proportion = 1 - train_proportion - val_proportion

n_train = round(len(data) * train_proportion)
n_val = round(len(data) * val_proportion)
n_test = len(data) - n_train - n_val

train_data, val_data, test_data = torch.utils.data.random_split(data, [n_train, n_val, n_test],
                                                                torch.Generator().manual_seed(1))
print(f'Train Size: {n_train}, Validation Size: {n_val}, Test Size: {n_test}')

Train Size: 62400, Validation Size: 3900, Test Size: 11700


In [18]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=256, shuffle=True)
model.train()
model.fit(train_loader, val_loader, epochs = 5, lr=1e-3)

  0%|          | 0/975 [00:00<?, ?it/s]/Users/claytonellis/Desktop/UCSB/Schoolwork/PSTAT 234/ASL Detection/utils.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(read_image(img_path),dtype=torch.float32)
/opt/anaconda3/envs/project_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1740: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|██████████| 16/16 [00:02<00:00,  5.60it/s]


Epoch 1: Training Loss = 3.1560678954002186
Epoch 1: Validation Loss = 3.0425856604942907


100%|██████████| 16/16 [00:02<00:00,  5.48it/s]


Epoch 2: Training Loss = 2.9758666926163895
Epoch 2: Validation Loss = 2.9089427654559796


100%|██████████| 16/16 [00:02<00:00,  5.41it/s]


Epoch 3: Training Loss = 2.8693943903996395
Epoch 3: Validation Loss = 2.8324422865647536


100%|██████████| 16/16 [00:02<00:00,  5.63it/s]


Epoch 4: Training Loss = 2.8138218576480183
Epoch 4: Validation Loss = 2.7839718940930487


100%|██████████| 16/16 [00:02<00:00,  5.62it/s]

Epoch 5: Training Loss = 2.7761076912513145
Epoch 5: Validation Loss = 2.752157221084986


In [28]:
model.eval()
all_preds = []
all_labels = []

test_loader = DataLoader(test_data, 64, True)
for inputs, outputs in test_loader:
    preds = model.predict(inputs.to(device))
    y_test = outputs.argmax(dim=2).squeeze()
    
    all_preds.append(preds)
    all_labels.append(y_test)

all_preds = torch.cat(all_preds).to('cpu')
all_labels = torch.cat(all_labels)


/Users/claytonellis/Desktop/UCSB/Schoolwork/PSTAT 234/ASL Detection/utils.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(read_image(img_path),dtype=torch.float32)
/opt/anaconda3/envs/project_env/lib/python3.11/site-packages/torch/nn/modules/module.py:1740: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:
# Accuracy.  The model is naive and barely trained, this isnt surprising
(all_preds == all_labels).sum() / all_preds.shape[0]

tensor(0.5573)